# LLama3

In [1]:
import pandas as pd
from huggingface_hub import login
login()


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

cuda


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [3]:
def generate_text_from_prompt(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=50, num_return_sequences=1, do_sample=True, top_k=0, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device)
# model.generation_config.pad_token_id = tokenizer.pad_token_id
generate_text_from_prompt("Give an example of a spam text")

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'Give an example of a spam text message you have received.\nGive an example of a spam text message you have received.\nI received a spam text message that read: "WIN A FREE IPAD! Text \'YES\' to 555-555'

In [8]:

def zs_data_file_path(task):
    return "/kaggle/working/"+task+"-zs-data-llama.csv"

def data_file_path(task):
    return "/kaggle/input/mini-llm-test/"+task+"-data-mini.csv"

def get_mid_file_path(task, mode = ""):
    return "/kaggle/working/mid-" + task + "-" + mode + "data.csv"

def count_req_rows(type):
    file = data_file_path(type)# '..\\data\\orig\\processed\\train\\news-data-mini.csv'
#     if(type=='sentiment'):
#         file = '..\\data\\orig\\processed\\train\\sentiment-data-mini.csv'
#     elif(type=='spam'):
#         file = '..\\data\\orig\\processed\\train\\spam-data-mini.csv'
    df = pd.read_csv(file)
    return int(df.shape[0]/2)

prompts = {
    'spam': 'Text message with advertisement or offer',
    'non_spam':'Text message from a friend or family says',
    'real_news':'Recently published political news title',
    'fake_news':'Fake political news title',
    'happy_tweet':'Tweet as a happy person',
    'sad_tweet': 'Tweet as a sad person'
}

zsl_tasks = [
    {
        'name':'news',
        'rows_per_category': count_req_rows('news'),
        'query': prompts['fake_news'],
        'non_query': prompts['real_news'],
        'print_count':10
    },
    {
        'name':'spam',
        'rows_per_category': count_req_rows('spam'),
        'query': prompts['spam'],
        'non_query': prompts['non_spam'],
        'print_count':100
    },    
    {
        'name':'sentiment',
        'rows_per_category': count_req_rows('sentiment'),
        'query': prompts['happy_tweet'],
        'non_query': prompts['sad_tweet'],
        'print_count':250
    }
]

def generate_text_from_prompt(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=50, num_return_sequences=1, do_sample=True, top_k=0, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


zsl_tasks


[{'name': 'news',
  'rows_per_category': 68,
  'query': 'Fake political news title',
  'non_query': 'Recently published political news title',
  'print_count': 10},
 {'name': 'spam',
  'rows_per_category': 602,
  'query': 'Text message with advertisement or offer',
  'non_query': 'Text message from a friend or family says',
  'print_count': 100},
 {'name': 'sentiment',
  'rows_per_category': 2499,
  'query': 'Tweet as a happy person',
  'non_query': 'Tweet as a sad person',
  'print_count': 250}]

In [9]:
df_main = pd.DataFrame()
for task in zsl_tasks:
    if task['name'] in ['news']:
        continue
    df_1 = pd.DataFrame(columns=['text'])
    df_2 = pd.DataFrame(columns=['text'])

    print("1. Starting task: ", task['name'])
    print("NQ-Progress:", end=' ')
    for _ in range(task['rows_per_category']):
        gen_text = generate_text_from_prompt(prompt = task['non_query'])
        df_2.loc[len(df_2)] = gen_text
        if _%task['print_count'] == 0:
            print(_, end=' ')
            df_2.to_csv(get_mid_file_path(task['name'], "n"), index=False)
    

    print("\n2. Generated non query df")
    print("Q-Progress:", end=' ')
    for _ in range(task['rows_per_category']):
        gen_text = generate_text_from_prompt(prompt = task['query'])
        df_1.loc[len(df_1)] = gen_text
        if _%task['print_count'] == 0:
            print(_, end=' ')
            df_1.to_csv(get_mid_file_path(task['name']), index=False)
    
    print("\n3. Generated query df")

    df_1['y']=1
    df_2['y']=0

    print("4. Saving df")

    df = pd.concat([df_1, df_2], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df_main = df
    df.to_csv(zs_data_file_path(task['name']), index=False)

    print("5. Ending task: ", task['name'])
    print()

1. Starting task:  spam
NQ-Progress: 0 100 200 300 400 500 600 
2. Generated non query df
Q-Progress: 0 100 200 300 400 500 600 
3. Generated query df
4. Saving df
5. Ending task:  spam

1. Starting task:  sentiment
NQ-Progress: 0 250 500 750 1000 1250 1500 1750 2000 2250 
2. Generated non query df
Q-Progress: 0 250 500 750 1000 1250 1500 1750 2000 2250 
3. Generated query df
4. Saving df
5. Ending task:  sentiment

